페이스북에서 개발한 FastText으로, Word2Vec의 확장 버전.

가장 큰 차이점은 Word2Vec은 단어를 더 이상 구분할 수 없는 최소단위로 본다면, FastText는 하나의 단어 안에 여러 단어들이 존재하는 것으로 간주함. 즉 서드워드(subword)를 고려함

### 1. 서브워드(subword) 학습
-------------------
FastText에서는 각 단어가 글자단위 n-gram의 구성을 취함. n을 몇으로 결정할지에 따라 단어들이 얼마나 분리되는지 결정됨. 


* 예를 들어 n을 3으로 잡은 tri-gram의 경우,

    * apple은 app, ppl, ple로 분리하고 ---> 이들을 벡터로 만듦
    * 정확하게는 시작과 끝을 의미하는 <, >를 도입하여 다음과 같이 토큰을 만들어 벡터화함

그리고 추가로 하나를 더 벡터화하는데, 전체 단어에 <, 와 >를 붙인 토큰임. 즉 다음과 같이 6개의 토큰을 벡터화함

실제 사용시에는 n의 최소값과 최대값 범위를 설정하는데, 기본값으로는 3과 6이 설정되어져 있음. 즉, 다음과 같은 결과가 나옴

이 서브워드들을 벡터화하는 것으로 이 단어에 대해 Word2Vec을 수행한다는 의미임. 위와 같이 내부단어들의 벡터값을 얻었다면 단어 apple의 벡터값은 위 벡터값들의 총합으로 구성됨

### 2. 모르는 단어(Out of Vocabulary, OOV)에 대한 대응
-----------------------------
FastText의 인공신경망을 학습한 후에 데이터셋의 모든 단어의 각 n-gram에 대해 워드임베딩이 됨. 이렇게 되면 장점은 데이터셋만 충분하다면 위와 같은 내부단어(subword)를 통해 모르는 단어에 대해서도 다른 단어와의 유사도를 계산할 수 있음


### 3. 단어집합 내 빈도수가 적은 단어(Rare Word)에 대한 대응
-------------------------------
Word2Vec의 경우, 등장빈도가 적은 단어(rare words)에 대해서는 임베딩 정확도가 높지 않다는 단점이 있음. 참고할 수 있는 경우의 수가 적다보니 정확하게 임베딩이 되지 않음

FastText의 경우, 단어가 희귀하더라도 비교적은 높은 임베딩 값을 얻을 수 있음. FastText가 노이즈가 ㅁ낳은 코퍼스에서 강점을 가지는 이유도 여기에 있음

모든 훈련 코퍼스에 오타나 맞춤법이 틀린 단어가 없으면 이상적이겠지만, 실제 많은 비정형 데이터에는 오타가 섞여 있음. 오타가 섞인 단어는 등장빈도가 적어 일정의 희귀단어가 됨. 

FastText에서는 이러한 단어들에도 대응 가능


### 4. Word2Vec vs. FastText 비교
-------------------------------
#### 1) Word2Vec

In [1]:
# 데이터 로딩 및 전처리
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x1a379886ac0>)

In [2]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='utf8')
target_text = etree.parse(targetXML)

In [3]:
# content 데이터만 추출
parse_text = '\n'.join(target_text.xpath('//content/text()'))

In [6]:
# 배경음 등 노이즈 제거
regex =  r'\([^)]*\)'
content_text = re.sub(regex, '', parse_text)

In [7]:
# 토큰화 수행
sent_text = sent_tokenize(content_text)

In [8]:
sent_text[0]

"Here are two reasons companies fail: they only do more of the same, or they only do what's new."

In [12]:
# 구두점 제거, 소문자로 변환
normalized_text = []
for string in sent_text:
    tokens = re.sub(r'[^a-z0-9]+', " ", string.lower())  # 빈칸으로 대체
    normalized_text.append(tokens)

In [13]:
# 각 문장에 대한 토큰화 수행
result = [word_tokenize(sentence) for sentence in normalized_text]

In [15]:
print(len(result))

273380


In [18]:
### word2vec 훈련시키기
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

model = Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=0)

In [19]:
## 모델 저장 및 로드
model.wv.save_word2vec_format('eng_w2v')
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v")

In [23]:
loaded_model.most_similar('electrofishing') # 단어가 없기 때문에 에러 발생

KeyError: "Key 'electrofishing' not present in vocabulary"

#### 2) FastText

In [24]:
from gensim.models import FastText

In [25]:
model = FastText(result, vector_size=100, window=5, min_count=5, workers=4, sg=1)  # sg: skip-gram

In [27]:
model.wv.most_similar("electrofishing")

[('electrolux', 0.8636344075202942),
 ('electrolyte', 0.8625998497009277),
 ('electroshock', 0.8513526320457458),
 ('electro', 0.8485394716262817),
 ('electroencephalogram', 0.8393594026565552),
 ('electrochemical', 0.8290709853172302),
 ('electrogram', 0.8269733190536499),
 ('electrons', 0.8167208433151245),
 ('electron', 0.8142461776733398),
 ('electric', 0.8126084208488464)]

Word2Vec에서는 유사한 단어를 찾아내지 못했으나, FastText는 유사한 단어를 계산해서 출력함